In [1]:
#Required libraries
import  datetime, time, smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import requests, json,csv
import pandas as pd 
from botocore.client import Config
import ibm_boto3
import ibm_db
#from urllib.request import urlopen

yesterday = datetime.date.today() - datetime.timedelta(1)
outyear = str(yesterday.year)
outmonth = str(yesterday.month)
outday = str(yesterday.day)
outdate =  outyear + outmonth.rjust(2, '0') + outday.rjust(2, '0')    

###############################################################################
#This method will call the 15 day hourly forecast for the specified latitude and longitude
#The forecast is then compared to the thresholds for wind speed and wind gusts.  If exceeded, an email using Gmail is triggered.
def APIcall(name, latitude, longitude, windThreshold, windGustThreshold, email): # function for pulling data   
    
    url = 'https://api.weather.com/v3/wx/forecast/hourly/15day?geocode=' + str(latitude) + ',' + str(longitude)
    url = url + '&format=json&units=e&language=en-US&apiKey=YOUR_TWC_KEY_HERE'         
    print ("Pulling Forecast data..." + time.strftime("%b %d,%Y %H:%M"))
    print (url)
    data = requests.get(url).json() #Our API Call
    #print(data)
    windGusts = data['windGust']    
    #need to go through all 360 hours and if any of the wind speed or wind gust
    #thresholds are broken, send an email with those above threshold and the entire 360 hour forecast as well.
    highWinds = []
    highWindGusts = []
    for x in range(0, 96):
        windGust = 0
        if data['windGust'][x] is not None:
            windGust = int(data['windGust'][x])
        wind = int(data['windSpeed'][x])
        date = data['validTimeLocal'][x]
        if(wind > int(windThreshold)):
            #print('WIND ALERT!: ' + name + ' wind = ' + str(wind) + ' wind Threshold = ' + windThreshold + ' wind gust = ' + str(windGust) + ' windGustThreshold = ' + windGustThreshold + ' forecast date and time: ' + str(date))
            windAlert = [name, wind, windThreshold, date, latitude, longitude]
            highWinds.append(windAlert)
        
        if(windGust > int(windGustThreshold)):
                #print('WIND GUST ALERT!: ' + name + ' wind = ' + str(wind) + ' wind Threshold = ' + windThreshold + ' wind gust = ' + str(windGust) + ' windGustThreshold = ' + windGustThreshold + ' forecast date and time: ' + str(date))
                windGustAlert = [name, windGust, windGustThreshold, date, latitude, longitude]
                highWindGusts.append(windGustAlert)
    print('*********************************** DONE ***************************************************')
    print('******************************** CHECKING FOR WIND ALERTS ***************************************')
    if(len(highWinds) > 0):
        print('We have ' + str(len(highWinds)) + ' wind alerts for this location')
        sub = name + ' | HIGH WIND ALERT! | 96 Hourly Alert Count:  ' + str(len(highWinds)) + ' | threshold = ' + str(windThreshold)
        bod = 'Client Location: ' + name + '(' + str(latitude) + ',' + str(longitude) + ')\nHigh Wind Speed Threshold: ' + str(windThreshold)
        bod = bod + '\nForecast Request Date (UTC): ' + time.strftime("%b %d,%Y %H:%M")
        bod = bod + '\nThe list below shows ' + str(len(highWinds)) + ' local hourly wind speed forecasts for the next 4 days (96 hours) that exceed threshold of ' + str(windThreshold) + '.'  
        bod = bod + '\n\nHourly Dates and Times Where Wind Speed Threshold of ' + str(windThreshold) + ' has been exceeded.'        
        for y in range(0, len(highWinds)):
            row = highWinds[y]
            yy = y + 1
            bod = bod + '\n' + str(yy) + ') ' + str(row[3]) + ' Forecast: ' + str(row[1]) +  ' Threshold: ' +  str(row[2])
        print("Subject: " + sub)
        print("Body: " + bod)
        send_email('YOUR_GMAIL_USERNAME_HERE', 'YOUR_GMAIL_PASSWORD_HERE', email, sub, bod)
    print('******************************** CHECKING FOR WIND GUST ALERTS ***************************************')
    if(len(highWindGusts) > 0):
        print('We have ' + str(len(highWindGusts)) + ' wind gust alerts for this location')
        sub = name + ' | HIGH WIND GUST ALERT! | 96 Hourly Alert Count:  ' + str(len(highWindGusts)) + ' | Wind Gust Threshold = ' + str(windGustThreshold)
        bod = 'Client Location: ' + name + '(' + str(latitude) + ',' + str(longitude) + ')\nHigh Wind Gust Threshold: ' + str(windGustThreshold)        
        bod = bod + '\nForecast Request Date (UTC): ' + time.strftime("%b %d,%Y %H:%M")
        bod = bod + '\nThe list below shows ' + str(len(highWindGusts)) + ' local hourly wind gust forecasts for the next 4 days (96 hours) that exceed threshold of ' + str(windGustThreshold) + '.'
        bod = bod + '\n\nHourly Dates and Times Where Wind Gust Threshold of ' + str(windGustThreshold) + ' has been exceeded.'        
        for z in range(0, len(highWindGusts)):
            row = highWindGusts[z]
            zz = z + 1
            bod = bod + '\n' + str(zz) + ') ' + str(row[3]) + ' Forecast: ' + str(row[1]) +  ' Threshold: ' +  str(row[2])
        print("Subject: " + sub)
        print("Body: " + bod)
        send_email('YOUR_GMAIL_USERNAME_HERE', 'YOUR_GMAIL_PASSWORD_HERE', email, sub, bod)    
def send_email(user, pwd, recipient, subject, body):
    FROM = user
    TO = recipient if isinstance(recipient, list) else [recipient]
    SUBJECT = subject
    TEXT = body

    # Prepare actual message
    message = """From: %s\nTo: %s\nSubject: %s\n\n%s
    """ % (FROM, ", ".join(TO), SUBJECT, TEXT)
    
    server = smtplib.SMTP("smtp.gmail.com", 587)
    server.ehlo()
    server.starttls()
    server.login(user, pwd)
    server.sendmail(FROM, TO, message)
    server.close()
    print ('successfully sent the mail to: ' + recipient)
    
# This is the section where we import the thresholds.  Each one in the database contains the name, latitude, longitude, windThreshold, windSpeedThreshold, and 
# email where to send alert if the forecast for that hour exceeds the threshold.  The email generated will contain all the hours where the thresholds have been exceeded.
###############################################################################
conn = ibm_db.connect("DATABASE=BLUDB;HOSTNAME=dashdb-entry-yp-dal09-08.services.dal.bluemix.net;PORT=50000;PROTOCOL=TCPIP;UID=YOUR_DB2_USERNAME_HERE;PWD=YOUR_DB2_PASSWORD_HERE;", "YOUR_DB2_USERNAME_HERE", "YOUR_DB2_PASSWORD_HERE")
results = []
if conn:
    sql = "SELECT * FROM ALERT_LIST_FOR_WIND"
    stmt = ibm_db.exec_immediate(conn, sql)
    result = ibm_db.fetch_both(stmt)
    while( result ):
        name = result[0]
        latitude = result[1]
        longitude = result[2]        
        windThreshold = result[3]
        windGustThreshold = result[4]
        email = result[5]
        
        print ("Result from Query:",name , latitude, longitude, windThreshold, windGustThreshold, email)
        results.append(result)
        result = ibm_db.fetch_both(stmt)
        
#Iterate through the list created from the database results.
for a in results:
    APIcall(a[0], a[1], a[2], a[3], a[4], a[5])

 

Exception: [IBM][CLI Driver] SQL30082N  Security processing failed with reason "24" ("USERNAME AND/OR PASSWORD INVALID").  SQLSTATE=08001 SQLCODE=-30082